In [1]:
import networkx as nx
import pickle
import random
import scipy
from scipy.sparse import tril, find
import django
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
import igraph

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from tmv_app.tasks import *

from django.db.models import Count, Sum

qid= 6230
q = Query.objects.get(pk=qid)

sample = 0

Cmat = scipy.sparse.load_npz(f'citation_data/bibCouple_q_{qid}_{sample}.npz')

with open(f"citation_data/docnet_dict_q_{qid}_{sample}.pickle", "rb") as f:
    m_dict = pickle.load(f)
with open(f"citation_data/docnet_revdict_q_{qid}_{sample}.pickle", "rb") as f:
    rev_m_dict = pickle.load(f)

Cmat.data *= Cmat.data>=1
Cmat.eliminate_zeros()

Cmat



<1974x1974 sparse matrix of type '<class 'numpy.int64'>'
	with 285004 stored elements in Compressed Sparse Row format>

In [2]:
def check_cites(bc,rev_m_dict, i=0):
    d1 = Doc.objects.get(pk=rev_m_dict[bc[0][i]])
    d2 = Doc.objects.get(pk=rev_m_dict[bc[1][i]])
    print(bc[2][i])
    return set(d2.wosarticle.cr) & set(d1.wosarticle.cr)

bc = find(tril(Cmat,k=-1))

check_cites(bc, rev_m_dict, 11)


3


{'Arab P, 2014, CHEM MATER, V26, P1385, DOI 10.1021/cm403161e',
 'Rabbani MG, 2012, CHEM COMMUN, V48, P1141, DOI 10.1039/c2cc16986j',
 'Suresh VM, 2014, ACS APPL MATER INTER, V6, P4630, DOI 10.1021/am500057z'}

In [41]:
## TODO read graph better, use find, get edge weights and 
def read_network(Cmat, rev_m_dict, doc_ids=None):
    g = igraph.Graph()
    g.add_vertices(len(rev_m_dict))
    edges = []
    weights = []
    n1s, n2s, ews = find(tril(Cmat,k=-1))
    for i, n1 in enumerate(n1s):
        if doc_ids:
            if rev_m_dict[n1] not in doc_ids or rev_m_dict[n2s[i]] not in doc_ids:
                continue
        e = (n1,n2s[i])
        edges.append(e)
        weights.append(ews[i])
    g.add_edges(edges)
    g.es["weight"] = weights

    g.vs["doc_id"] = list(rev_m_dict.values())
    return g
    pass

# We can filter here, if we only want to see an early part of the graph
docs = Doc.objects.filter(pk__in=list(rev_m_dict.values())).filter(PY__lt=2017)
doc_ids = docs.values_list('pk',flat=True)
print(docs.count())

g = read_network(Cmat, rev_m_dict, doc_ids)
print(g.summary())

1309
IGRAPH U-W- 1974 63978 -- 
+ attr: doc_id (v), weight (e)


In [42]:
for c in g.clusters():
    print(len(c))

1264
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

In [43]:
g = g.clusters().giant()

In [ ]:
from fa2 import ForceAtlas2

forceatlas2 = ForceAtlas2(
    # Behavior alternatives
    outboundAttractionDistribution=True,  # Dissuade hubs
    linLogMode=False,  # NOT IMPLEMENTED
    adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
    edgeWeightInfluence=1,

    # Performance
    jitterTolerance=1.0,  # Tolerance
    barnesHutOptimize=True,
    barnesHutTheta=1.2,
    multiThreaded=False,  # NOT IMPLEMENTED

    # Tuning
    scalingRatio=2.0,
    strongGravityMode=False,
    gravity=1,

    # Log
    verbose=True
)

positions = forceatlas2.forceatlas2_igraph_layout(g, pos=None, iterations=2000, weight_attr="weight")

In [ ]:
clusters = g.community_fastgreedy(weights="weight")
cl = clusters.as_clustering()


In [ ]:
def get_text(df):
    t = "\n".join([f"{r['query__category__name']} - {r['pc']:.0%}" for i,r in df.iterrows() if r['pc']>0.1])
    return t

In [ ]:

                   
def draw_network(ax, g, layout, labels, interactive=False, clusters=None, rev_m_dict=None, pos="top"):
    #label_ids = [l['id'] for l in labels]
    ax.grid(False)
    ax.axis('off')
    
    x_offset=0
    if clusters:
        if rev_m_dict:
            pass
            
        for c in clusters:
            x = [x[0] for i,x in enumerate(layout) if i in c]
            y = [x[1] for i,x in enumerate(layout) if i in c] 
            scatter = ax.scatter(x,y,edgecolor="black")
            
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        xrange = xlim[1] - xlim[0]
        yrange = ylim[1] - ylim[0]
        x_step = xrange/3
        if pos=="top":
            ha = "center"
            x_offset = x_step/2
        else:
            ha="left"
            x_offset = xrange
        y_offset = 0
        texts = []
        for i,c in enumerate(clusters):
            x = [x[0] for i,x in enumerate(layout) if i in c]
            y = [x[1] for i,x in enumerate(layout) if i in c] 
            scatter = ax.scatter(x,y,edgecolor="black")
            col = scatter.get_facecolors()[0]
            cx, cy = sum(x) / len(x), sum(y) / len(y)
            ids = [rev_m_dict[x] for x in c]
            #docs = Doc.objects.filter(query=qid,pk__in=ids)
            dos = DocOwnership.objects.filter(
                doc__id__in=ids,
                query__project=q.project,
                relevant=1
            ).values('query__category__name').annotate(
                n = Count('pk')
            ).order_by('-n')
            df = pd.DataFrame.from_dict(list(dos))
            #df = pd.DataFrame.from_dict(list(docs.values('doccat__category__name').annotate(
            #    n=Count('pk')
            #).order_by('-n')))
            df['pc'] = round(df['n'] / df['n'].sum(),2)
            t = f"Cluster {i} - {len(ids)} docs\n\n{get_text(df)}"
            text = ax.text(
                    xlim[0]+x_offset,
                    ylim[1]+y_offset,
                    t,
                    bbox={'facecolor':col, 'alpha':0.5, 'pad':0.4, 'boxstyle': 'round'},
                    va="top",
                    ha=ha
                )
            texts.append(text)
            if pos=="top":
                x_offset+=x_step
                if xlim[0]+x_offset > xlim[1]:
                    fig.canvas.draw()
                    bs = []
                    for t in texts:
                        box = t.get_bbox_patch().get_window_extent()
                        tcbox = ax.transData.inverted().transform(box)
                        bs.append(tcbox[0][1])

                    y_offset+=min(bs)-ylim[1]
                    x_offset = x_step/2
            else:
                fig.canvas.draw()
                box = text.get_bbox_patch().get_window_extent()
                tcbox = ax.transData.inverted().transform(box)
                b = tcbox[0][1]

                y_offset=b-ylim[1] -50             
            
            
            #ax.text(cx, cy, "ASDFASDFASFD")
    else:
        x = [x[0] for x in layout]
        y = [x[1] for x in layout]
        scatter = ax.scatter(x,y,edgecolor="black")
    
fig, ax = plt.subplots(figsize=(12,12),edgecolor="black",frameon=True,linewidth=4)

draw_network(ax, g, positions, [], clusters=cl, rev_m_dict=rev_m_dict,pos="right")
#draw_network(ax, g, positions, [])

fig.tight_layout()

plt.savefig(f"plots/citation_networks/bibgraph_couple_forceatlas_fastgreedy_{qid}.svg")
plt.savefig(
    f"plots/citation_networks/bibgraph_couple_forceatlas_fastgreedy_{qid}.png",
    bbox_inches="tight"
)

plt.show()



In [10]:
# Have a look at some of the 
for c in cl:
    pass
ids = [rev_m_dict[x] for x in c]
docs = Doc.objects.filter(query=qid,pk__in=ids)
df = pd.DataFrame.from_dict(list(docs.values('doccat__category__name').annotate(
    n=Count('pk')
).order_by('-n')))
df['pc'] = round(df['n'] / df['n'].sum(),2)

df.head()

for d in docs:
    print(d.title)
    print("; ".join(d.doccat_set.values_list('category__name',flat=True)))

Construction Materials from Stainless Steel Slags: Technical Aspects, Environmental Benefits, and Economic Opportunities
Other mineralisation products; Carbonates production; Other mineralisation chemicals (no carbonates)
Enhancement of accelerated carbonation of alkaline waste residues by ultrasound
Cement/concrete production; Other mineralisation products
Electrochemical synthesis of fuels by CO2 hydrogenation on Cu in a potassium ion conducting membrane reactor at bench scale
Methanol; DME
Power to liquid and power to gas: An option for the German Energiewende
Methanol
Calcium silicates synthesised from industrial residues with the ability for CO2 sequestration
Cement/concrete production; Other mineralisation products
Identification of early opportunities for CO2 sequestration-worldwide screening for CO2-EOR and CO2-ECBM projects
EOR
Deployment of Accelerated Carbonation Using Alkaline Solid Wastes for Carbon Mineralization and Utilization Toward a Circular Economy
Cement/concrete p

In [11]:
ids = [rev_m_dict[x] for x in c]
docs = Doc.objects.filter(query=qid,pk__in=ids)
df = pd.DataFrame.from_dict(list(docs.values('doccat__category__name').annotate(
    n=Count('pk')
).order_by('-n')))
df['pc'] = round(df['n'] / df['n'].sum(),2)
df.head()

,doccat__category__name,n,pc
0,Other mineralisation products,9,0.32
1,Cement/concrete production,6,0.21
2,Carbonates production,3,0.11
3,Biofuel from algae,3,0.11
4,Methanol,2,0.07


In [12]:
ids = [rev_m_dict[x] for x in c]
docs = Doc.objects.filter(query=qid,pk__in=ids)
dos = DocOwnership.objects.filter(
    doc__id__in=ids,
    query__project=q.project
).values('query__category__name').annotate(
    n = Count('pk')
).order_by('-n')
print(dos)

<QuerySet [{'query__category__name': 'Cement/concrete production', 'n': 6}, {'query__category__name': 'Other mineralisation products', 'n': 3}, {'query__category__name': 'Biofuel from algae', 'n': 3}, {'query__category__name': 'Methanol', 'n': 2}, {'query__category__name': 'EOR', 'n': 2}, {'query__category__name': 'DME', 'n': 1}, {'query__category__name': 'Acetic acid', 'n': 1}]>


In [13]:
q.project.id

106